In [17]:
import collections
import os
from pathlib import Path

clusters = [2000, 500, 500, 500, 500, 1000, 500, 750, 1000]

current_dir = os.getcwd()
with open(f'{current_dir}/ground_truth/clustering', 'w') as f:
    cluster_idx = 0
    for cluster in clusters:
        for i in range(cluster):
            f.write(f'{cluster_idx},')
        cluster_idx+=1
print(f'GT clustering written to {current_dir}/ground_truth/clustering')

GT clustering written to /Users/dd/work/secedo-experiments/varsim/robustness/ground_truth/clustering


In [52]:
import collections

def get_dominant_cluster(d_clusters, start, stop):
    counters: dict = collections.Counter(d_clusters[start:stop])
    max_v = 0
    max_k = 0
    for k,v in counters.items():
        if max_v < v:
            max_v = v
            max_k = k
    return int(max_k)


input_files = [f'{x}/clustering' for x in Path('.').iterdir() if str(x).startswith('h0')]
print(input_files)
input_files.sort()
for input_file in input_files:
    #print(f'Processing {input_file}')
    if not os.path.exists(input_file):
        # print(f'Warning: {input_file} does not exist')
        continue
    d_clusters = [int(v) for v in open(input_file).read().split(',')]
    #print(d_clusters)
    d_idx_to_idx = {}
    detected_idx = []
    d_clusters_idx = []
    start = 0
    #print(len(d_clusters))
    for idx, cluster in enumerate(clusters):
        d_idx = get_dominant_cluster(d_clusters, start, start+cluster)
        if not d_idx in d_idx_to_idx:
            d_idx_to_idx[d_idx] = idx
        #print(f'Mapping [{start}:{start+cluster}] {d_idx} to {idx}')
        detected_idx.append(d_idx_to_idx[d_idx])
        start += cluster

    canonical_clusters = [d_idx_to_idx[v] if v in d_idx_to_idx else v+100 for v in d_clusters]
    counts = collections.Counter(canonical_clusters)
    # precision & recall
    start = 0
    h = input_file[1:4]
    t_idx = input_file.index('theta')
    theta = input_file[t_idx + 5: input_file.index('_', t_idx)]
    e_idx = input_file.index('epsilon')
    eps = input_file[e_idx+len('epsilon'): input_file.index('/')]
    print(f'$h={h}, \\theta={theta}, \\epsilon={eps}$', end = ' & ')
    for idx, cluster in enumerate(clusters):
        tp = canonical_clusters[start:start+cluster].count(idx)
        print(f'{tp/counts[idx]:.2f} {tp/cluster:.2f}', end=' & ' if idx < len(clusters) - 1 else ' ')
        start += cluster
    print('\\\\')

    #print(canonical_clusters)






['h0.5_theta0.05_epsilon0.05/clustering', 'h0.3_theta0.005_epsilon0.01/clustering', 'h0.5_theta0.01_epsilon0.01/clustering', 'h0.8_theta0.05_epsilon0.01/clustering', 'h0.8_theta0.01_epsilon0.05/clustering', 'h0.5_theta0.005_epsilon0.01/clustering', 'h0.3_theta0.05_epsilon0.05/clustering', 'h0.3_theta0.01_epsilon0.01/clustering', 'h0.8_theta0.005_epsilon0.01/clustering', 'h0.5_theta0.05_epsilon0.01/clustering', 'h0.5_theta0.01_epsilon0.05/clustering', 'h0.3_theta0.005_epsilon0.05/clustering', 'h0.8_theta0.05_epsilon0.05/clustering', 'h0.8_theta0.01_epsilon0.01/clustering', 'h0.5_theta0.005_epsilon0.05/clustering', 'h0.3_theta0.05_epsilon0.01/clustering', 'h0.3_theta0.01_epsilon0.05/clustering', 'h0.8_theta0.005_epsilon0.05/clustering']
$h=0.3, \theta=0.005, \epsilon=0.01$ & 1.00 0.50 & 0.97 0.98 & 1.00 1.00 & 1.00 1.00 & 1.00 1.00 & 0.96 0.96 & 1.00 1.00 & 0.99 0.98 & 0.96 0.96 \\
$h=0.3, \theta=0.005, \epsilon=0.05$ & 1.00 0.97 & 0.87 1.00 & 1.00 1.00 & 1.00 0.97 & 1.00 0.98 & 0.94 0.9